In [1]:
import requests
from langchain_core.documents import Document
from langchain.chains import RetrievalQA
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.llms import Ollama
from langchain.vectorstores.pgvector import PGVector
from dotenv import load_dotenv
import os

def fetch_github_issues(owner="langflow-ai", repo="langflow",token=None):
    """
    Fetch issues from a GitHub repository using requests
    :param owner: Repository owner/organization
    :param repo: Repository name
    """
    # GitHub API URL for issues
    url = f"https://api.github.com/repos/{owner}/{repo}/issues"
    
    # Add headers to avoid rate limiting issues
    headers = {
        'Accept': 'application/vnd.github.v3+json',
        'User-Agent': 'Python-GitHub-Issues-Example'
    }
    if github_token:
        headers['Authorization'] = f'token {github_token}'
    
    try:
        # Make the GET request
        response = requests.get(url, headers=headers)
        
        # Check if the request was successful
        if response.status_code == 200:
            issues = response.json()
            
            print(f"\nFetching issues from {owner}/{repo}:")
            print("-" * 50)
            docs = []
            # Print each issue
            for issue in issues:
                metadata = {
                    "author": issue["user"]["login"],
                    "comments": issue["comments"],
                    "body": issue["body"],
                    "labels": issue["labels"],
                    "created_at": issue["created_at"],
                    }
                data = issue["title"]
                if issue["body"]:
                    data += issue["body"]
                doc = Document(page_content=data, metadata=metadata)
                docs.append(doc)
                
        else:
            print(f"Failed to fetch issues. Status code: {response.status_code}")
            print(f"Error message: {response.text}")
        
        return docs
            
    except Exception as e:
        print(f"An error occurred: {str(e)}")
    


In [2]:
def connect_to_vector_db():
    try:
        # Load environment variables
        load_dotenv()
        
        # Get connection string
        CONNECTION_STRING = os.getenv('DATABASE_URL')
        if not CONNECTION_STRING:
            raise ValueError("DATABASE_URL environment variable not found")
            
        # Initialize embeddings
        ollama_embeddings = OllamaEmbeddings(
            model="mxbai-embed-large:latest",
            temperature=0.1
        )

        COLLECTION_NAME = 'github_issues'  # Fixed typo in collection name
        
        # Create vector store
        vector_store = PGVector(
            connection_string=CONNECTION_STRING,
            collection_name=COLLECTION_NAME,
            embedding_function=ollama_embeddings
        )
        
        return vector_store
        
    except Exception as e:
        print(f"Error connecting to vector database: {str(e)}")
        return None

In [3]:
def load_data_to_vector_db(githubIssuesData):

    ollama_embeddings = OllamaEmbeddings(
        model="llama2-uncensored:latest",
        temperature=0.1
    )

    CONNECTION_STRING = os.getenv('DATABASE_URL')
    COLLECTION_NAME = 'github_issues'
    vector_store = PGVector.from_documents(
    embedding=OllamaEmbeddings(model="mxbai-embed-large"),
    documents=githubIssuesData,
    collection_name=COLLECTION_NAME,
    connection_string=CONNECTION_STRING)

    return vector_store

In [4]:
load_dotenv()
github_token = os.getenv('GITHUB_TOKEN')
# Fetch issues


# Connect to vector database
vector_store = connect_to_vector_db()


add_to_vectorstore = input("Do you want to update the issues? (y/N): ").lower() in [
    "yes",
    "y",
]

if add_to_vectorstore:
    owner="langflow-ai"
    repo="langflow"
    vector_store.delete_collection()
    issuesData = fetch_github_issues(owner=owner, repo=repo,token=github_token)
    # Add issues to vector database 
    vector_store = load_data_to_vector_db(issuesData)
else:
    vector_store = connect_to_vector_db()


C:\Users\saigi\AppData\Local\Temp\ipykernel_20412\1659217320.py:12: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  ollama_embeddings = OllamaEmbeddings(
C:\Users\saigi\AppData\Local\Temp\ipykernel_20412\1659217320.py:20: LangChainPendingDeprecationWarning: This class is pending deprecation and may be removed in a future version. You can swap to using the `PGVector` implementation in `langchain_postgres`. Please read the guidelines in the doc-string of this class to follow prior to migrating as there are some differences between the implementations. See <https://github.com/langchain-ai/langchain-postgres> for details about the new implementation.
  vector_store = PGVector(
C:\U

In [5]:
def test_similarity_search(vector_store, query="What are the issues related to Langflow crashing?", k=2):
    try:
        if vector_store is None:
            raise ValueError("Vector store not initialized")
            
        # Perform similarity search
        results = vector_store.similarity_search_with_score(query, k=k)
        
        print(f"\nSearch Query: {query}")
        print("-" * 50)
        
        for doc, score in results:
            print(f"\nScore: {score:.4f}")
            print(f"Content: {doc.page_content[:200]}...")  # Show first 200 chars
            print(f"Metadata: Created at {doc.metadata.get('created_at', 'N/A')}")
            print("-" * 50)
            
        return results
        
    except Exception as e:
        print(f"Error during similarity search: {str(e)}")
        return None

In [6]:
test_similarity_search(vector_store)


Search Query: What are the issues related to Langflow crashing?
--------------------------------------------------

Score: 0.2305
Content: LangFlow suddenly crashed and can no longer be started### Bug Description

Yesterday, I started LangFlow inside a container and created two Vector Store RAG workflows:
flow_1: Vectorized Markdown d...
Metadata: Created at 2024-11-20T03:39:20Z
--------------------------------------------------

Score: 0.2850
Content: After upgrade to version 1.1.0, the old flows fail to build because components are out of date### Bug Description

The problem here is that quite a few components have been upgraded in the new version...
Metadata: Created at 2024-11-20T15:30:38Z
--------------------------------------------------


[(Document(metadata={'author': 'tzengshinfu', 'comments': 3, 'body': "### Bug Description\r\n\r\nYesterday, I started LangFlow inside a container and created two Vector Store RAG workflows:\r\nflow_1: Vectorized Markdown documents and wrote them into PGVector.\r\nflow_2: Matched user queries with PGVector to retrieve answers.\r\nBoth workflows ran smoothly yesterday.\r\n\r\nThis morning, I discovered that LangFlow had suddenly crashed, and the log is as follows:\r\n[langflow-crash.log](https://github.com/user-attachments/files/17824375/langflow-crash.log)(As seen at [11/20/24 09:39:20])\r\n\r\nEven after deleting the vectorization records in PGVector, it still couldn't start successfully.\r\nUltimately, I had to delete all the tables storing the vectorized document data to get it to start properly.\r\n\r\n### Reproduction\r\n\r\n1. Start LangFlow and PGVector (container settings are as shown in the attachment: [docker-compose.yml.txt](https://github.com/user-attachments/files/17824361/

In [7]:
from langchain_community.llms import Ollama
## Load Ollama llam3 LLM model
llama3_llm=Ollama(model="llama3.1:latest")
llama3_llm

C:\Users\saigi\AppData\Local\Temp\ipykernel_20412\2579645501.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llama3_llm=Ollama(model="llama3.1:latest")


Ollama(model='llama3.1:latest')

In [8]:
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor
from langchain.tools.retriever import create_retriever_tool

In [9]:
retriever = vector_store.as_retriever(search_kwargs={"k": 3})
retriever_tool = create_retriever_tool(
    retriever,
    "github_search",
    "Search for information about github issues. For any questions about github issues, you must use this tool!",
)

In [10]:
from langchain_core.tools import tool


@tool
def note_tool(note):
    """
    saves a note to a local file

    Args:
        note: the text note to save
    """
    with open("notes.txt", "a") as f:
        f.write(note + "\n")


In [11]:
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.prompts import PromptTemplate

tools = [retriever_tool, note_tool]
# Create a prompt template for ReAct agent
prompt = PromptTemplate.from_template("""You are a helpful AI assistant that helps users find information about GitHub issues.

You have access to the following tools:

{tools}


Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Question: {input}
{agent_scratchpad}""")

# Create the ReAct agent
agent = create_react_agent(llama3_llm, tools, prompt)

# Create the executor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Interactive loop
while (question := input("Ask a question about github issues (q to quit): ")) != "q":
    try:
        result = agent_executor.invoke({"input": question})
        print(result["output"])
    except Exception as e:
        print(f"An error occurred: {str(e)}")



> Entering new AgentExecutor chain...
Thought: I need to find information about GitHub issues related to LiteLLM Connector.
Action: github_search
Action Input: "LiteLLM Connector"Add LiteLLM Connector### Feature Request

I would like to suggest adding a connector for LiteLLM to LangFlow.
LiteLLM is a lightweight and fast open-source library for integrating various LLM providers with a simplified interface. Including this connector in LangFlow would offer several benefits:

Broader compatibility: Support for multiple LLM providers through LiteLLM's unified interface.

Improved usability: Simplifies the process of integrating and switching between LLMs.

Community adoption: LiteLLM has an active and growing user base, making it a valuable addition to LangFlow.


Proposed Solution

1. Create a new connector module in LangFlow that interfaces with LiteLLM.
2. Ensure compatibility with key LiteLLM features, such as provider switching and request optimization.
3. Provide documentation and 